# Fine-tuning

In the previous section, we went through how to construct training and testing data properly. In this tutorial, we will actually fine-tune the model.

## Installation

Note to fine-tune BGE models using FlagEmbedding, we need to install the package with the finetune dependency:

In [ ]:
% pip install -U FlagEmbedding[finetune]

## Fine-tune

Below are the arguments for fine-tuning:

The following arguments are for model:
- `model_name_or_path`: The model checkpoint for initialization.
- `config_name`: Pretrained config name or path if not the same as model_name.
- `tokenizer_name`: Pretrained tokenizer name or path if not the same as model_name.
- `cache_dir`: Where do you want to store the pre-trained models downloaded from s3.
- `trust_remote_code`: Trust remote code
- `token`: The token to use when accessing the model.

The following arguments are for data:
- `train_data`: One or more paths to training data. `query: str`, `pos: List[str]`, `neg: List[str]` are required in the training data. Argument type: multiple.
- `cache_path`: Where do you want to store the cached data.
- `train_group_size`: (No metadata provided)
- `query_max_len`: The maximum total input sequence length after tokenization for passage. Sequences longer than this will be truncated.
- `passage_max_len`: The maximum total input sequence length after tokenization for passage. Sequences longer than this will be truncated.
- `pad_to_multiple_of`: If set will pad the sequence to be a multiple of the provided value.
- `max_example_num_per_dataset`: The max number of examples for each dataset.
- `query_instruction_for_retrieval`: Instruction for query.
- `query_instruction_format`: Format for query instruction.
- `knowledge_distillation`: Use knowledge distillation when `pos_scores: List[float]` and `neg_scores: List[float]` are in features of training data.
- `passage_instruction_for_retrieval`: Instruction for passage.
- `passage_instruction_format`: Format for passage instruction.
- `shuffle_ratio`: The ratio of shuffling the text.
- `same_dataset_within_batch`: All samples in the same batch comes from the same dataset.
- `small_threshold`: The threshold of small dataset. All small dataset in the same directory will be merged into one dataset.
- `drop_threshold`: The threshold for dropping merged small dataset. If the number of examples in the merged small dataset is less than this threshold, it will be dropped.

And the following extra arguments:
- `negatives_cross_device`: Share negatives across devices.
- `temperature`: Temperature used for similarity score.
- `fix_position_embedding`: Freeze the parameters of position embeddings.
- `sentence_pooling_method`: The pooling method. Available options: cls, mean, last_token. Default: cls.
- `normalize_embeddings`: Whether to normalize the embeddings.
- `sub_batch_size`: Sub batch size for training.
- `kd_loss_type`: The loss type for knowledge distillation. Available options: kl_div, m3_kd_loss. Default: kl_div.

In [1]:
%%bash
torchrun --nproc_per_node 2 \
	-m FlagEmbedding.finetune.embedder.encoder_only.base \
	--model_name_or_path BAAI/bge-large-en-v1.5 \
    --cache_dir ./cache/model \
    --train_data ./ft_data/training.json \
    --cache_path ./cache/data \
    --train_group_size 8 \
    --query_max_len 512 \
    --passage_max_len 512 \
    --pad_to_multiple_of 8 \
    --query_instruction_for_retrieval 'Represent this sentence for searching relevant passages: ' \
    --query_instruction_format '{}{}' \
    --knowledge_distillation False \
	--output_dir ./test_encoder_only_base_bge-large-en-v1.5 \
    --overwrite_output_dir \
    --learning_rate 1e-5 \
    --fp16 \
    --num_train_epochs 2 \
    --per_device_train_batch_size 2 \
    --dataloader_drop_last True \
    --warmup_ratio 0.1 \
    --gradient_checkpointing \
    --deepspeed config/ds_stage0.json \
    --logging_steps 1 \
    --save_steps 1000 \
    --negatives_cross_device \
    --temperature 0.02 \
    --sentence_pooling_method cls \
    --normalize_embeddings True \
    --kd_loss_type kl_div

W1223 06:27:06.807000 1362426 site-packages/torch/distributed/run.py:793] 
W1223 06:27:06.807000 1362426 site-packages/torch/distributed/run.py:793] *****************************************
W1223 06:27:06.807000 1362426 site-packages/torch/distributed/run.py:793] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1223 06:27:06.807000 1362426 site-packages/torch/distributed/run.py:793] *****************************************
/share/project/xzy/Envs/ft/lib/python3.11/site-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-depre

[2024-12-23 06:27:31,423] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-12-23 06:27:31,424] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-12-23 06:27:40,529] [INFO] [comm.py:652:init_distributed] cdb=None
[2024-12-23 06:27:40,529] [INFO] [comm.py:652:init_distributed] cdb=None
[2024-12-23 06:27:40,529] [INFO] [comm.py:683:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


12/23/2024 06:27:40 - WARNING - FlagEmbedding.abc.finetune.embedder.AbsRunner -   Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: True
12/23/2024 06:27:40 - INFO - FlagEmbedding.abc.finetune.embedder.AbsRunner -   Training/evaluation parameters AbsEmbedderTrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=True,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
de

[1734935704.354551] [job-40fb0ce3-8bfb-46ea-b409-0a2e2a1a3163-master-0:1362491:f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device
[1734935704.383634] [job-40fb0ce3-8bfb-46ea-b409-0a2e2a1a3163-master-0:1362492:f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device


Using /root/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
Using /root/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu124/fused_adam/build.ninja...
/share/project/xzy/Envs/ft/lib/python3.11/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module fused_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


...

 95%|█████████▌| 3000/3150 [12:29<00:34,  4.40it/s]12/23/2024 06:47:36 - INFO - FlagEmbedding.finetune.embedder.encoder_only.base.trainer -   Saving model checkpoint to ./test_encoder_only_base_bge-large-en-v1.5/checkpoint-3000


{'loss': 0.0684, 'grad_norm': 7.017930091803245, 'learning_rate': 5.361552028218695e-07, 'epoch': 1.91}
{'loss': 0.0, 'grad_norm': 0.000511339045166218, 'learning_rate': 5.326278659611994e-07, 'epoch': 1.91}
{'loss': 0.0, 'grad_norm': 0.0005950045300509824, 'learning_rate': 5.291005291005291e-07, 'epoch': 1.91}
{'loss': 0.0, 'grad_norm': 0.003924217893271839, 'learning_rate': 5.255731922398589e-07, 'epoch': 1.91}
{'loss': 0.0, 'grad_norm': 0.004307797663678439, 'learning_rate': 5.220458553791887e-07, 'epoch': 1.91}
{'loss': 0.0, 'grad_norm': 2.603897655768594e-05, 'learning_rate': 5.185185185185186e-07, 'epoch': 1.91}
{'loss': 0.0, 'grad_norm': 5.483371389463175e-06, 'learning_rate': 5.149911816578484e-07, 'epoch': 1.91}
{'loss': 0.4844, 'grad_norm': 20.3684123804579, 'learning_rate': 5.114638447971781e-07, 'epoch': 1.91}
{'loss': 0.0001, 'grad_norm': 0.01247325742979955, 'learning_rate': 5.07936507936508e-07, 'epoch': 1.91}
{'loss': 0.0004, 'grad_norm': 0.05214513400326202, 'learning_

100%|██████████| 3150/3150 [13:10<00:00,  3.72it/s]12/23/2024 06:48:17 - INFO - FlagEmbedding.finetune.embedder.encoder_only.base.trainer -   Saving model checkpoint to ./test_encoder_only_base_bge-large-en-v1.5/checkpoint-3150


{'train_runtime': 799.0537, 'train_samples_per_second': 15.769, 'train_steps_per_second': 3.942, 'train_loss': 0.04348497095562163, 'epoch': 2.0}


100%|██████████| 3150/3150 [13:19<00:00,  3.94it/s]
12/23/2024 06:48:26 - INFO - FlagEmbedding.finetune.embedder.encoder_only.base.trainer -   Saving model checkpoint to ./test_encoder_only_base_bge-large-en-v1.5
[rank0]:[W1223 06:48:28.948814944 ProcessGroupNCCL.cpp:1250] Warning: WARNING: process group has NOT been destroyed before we destruct ProcessGroupNCCL. On normal program exit, the application should call destroy_process_group to ensure that any pending NCCL operations have finished in this process. In rare cases this process can exit before this point and block the progress of another member of the process group. This constraint has always been present,  but this warning has only been added since PyTorch 2.4 (function operator())
